In [18]:
import os
import pandas
import psycopg2
import datetime

import psycopg2.extras

from sqlalchemy import create_engine

In [2]:
## Getting data from DGE's file

df = pd.read_csv(
    '../data/220817COVID19MEXICO.tar.gz', 
    compression='gzip', 
    error_bad_lines=False,
    usecols=['FECHA_SINTOMAS', 'FECHA_DEF', 'CLASIFICACION_FINAL'])
df.shape

<IPython.core.display.Javascript object>

(18006615, 3)

In [3]:
## Processing DGE data

#df[df['CLASIFICACION_FINAL'].isna()].head()
df.dropna(axis=0, subset=['CLASIFICACION_FINAL'], inplace=True)
df['CLASIFICACION_FINAL'] = df['CLASIFICACION_FINAL'].astype(int)
df.head()

,FECHA_SINTOMAS,FECHA_DEF,CLASIFICACION_FINAL
0,2020-11-30,9999-99-99,3
1,2021-01-13,9999-99-99,7
2,2020-02-18,9999-99-99,6
3,2020-04-18,9999-99-99,7
4,2020-07-20,9999-99-99,3


In [4]:
## Reading DB credentials

dbname = os.environ['DBNAME']
dbport = os.environ['DBPORT']
dbhost = os.environ['DBHOST']
dbuser = os.environ['DBUSER']
dbpass = os.environ['DBPASS']

In [17]:
## Getting covariables from Epi-PUMA DB

connection = psycopg2.connect(
    host=dbhost,
    port=dbport,
    database=dbname,
    user=dbuser,
    password=dbpass)
cursr = connection.cursor(cursor_factory = psycopg2.extras.RealDictCursor)

cursr.execute("SELECT id, clasificacion_final FROM covariable")
results = cursr.fetchall()

for covariable in results:
    print(covariable)
    
cursr.close()
connection.close()

RealDictRow([('id', 1), ('clasificacion_final', 'NEGATIVO A SARS-COV-2 POR LABORATORIO')])
RealDictRow([('id', 2), ('clasificacion_final', 'CASO DE COVID-19 CONFIRMADO POR ASOCIACIÓN CLÍNICA EPIDEMIOLÓGICA')])
RealDictRow([('id', 3), ('clasificacion_final', 'CASO DE SARS-COV-2  CONFIRMADO POR LABORATORIO')])
RealDictRow([('id', 4), ('clasificacion_final', 'NO REALIZADO POR LABORATORIO')])
RealDictRow([('id', 5), ('clasificacion_final', 'CASO DE COVID-19 CONFIRMADO POR COMITÉ DE  DICTAMINACIÓN')])
RealDictRow([('id', 6), ('clasificacion_final', 'INVÁLIDO POR LABORATORIO')])
RealDictRow([('id', 7), ('clasificacion_final', 'CASO SOSPECHOSO')])


In [ ]:
## Getting data from Epi-PUMA DB

infected = [5, 2, 3, 7]

sql = "SELECT date_occurrence, fecha_def, clasificacion_final \
        FROM occurrence where covariable_id in ({0})"\
        .format(','.join([str(strid) for strid in infected]))

engine = create_engine('postgresql://{0}:{1}@{2}:{3}/{4}'.format(
    dbuser,
    dbpass,
    dbhost,
    dbport,
    dbname
))

dfdb = pd.read_sql(sql, engine)
dfdb.head()

<IPython.core.display.Javascript object>